In [1]:
# install openmodelica from its repository
! for deb in deb deb-src; do echo "$deb http://build.openmodelica.org/apt `lsb_release -cs` release"; done | sudo tee /etc/apt/sources.list.d/openmodelica.list
! wget -q http://build.openmodelica.org/apt/openmodelica.asc -O- | sudo apt-key add -
! apt update && apt install openmodelica

deb http://build.openmodelica.org/apt bionic release
deb-src http://build.openmodelica.org/apt bionic release
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 http://build.openmodelica.org/apt bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://build.openmodelica.org/apt bionic Release [26.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http

In [2]:
# install Funz through pypi
!pip install --upgrade https://github.com/Funz/Funz.py/tarball/master
!#pip install Funz
import Funz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 52.3 MB 2.3 MB/s
     |████████████████████████████████| 200 kB 7.6 MB/s 
  Created wheel for Funz: filename=Funz-1.15.post6-py3-none-any.whl size=52354599 sha256=6f9b12112228cfe51d5e4556716b775ada4ebd673786f470088a6898413ddef6
  Stored in directory: /tmp/pip-ephem-wheel-cache-eznkr_l5/wheels/8a/2d/28/10fc13951eca236a2bbc92e357b62512b88a8ed6ab207be6d6
Successfully built Funz


In [3]:
# install plugin to support Modelica I/O
Funz.installModel('Modelica', edit_script=False)

...........Installed Funz model Modelica
Funz model Modelica added.


In [4]:
# setup main input file
%%writefile NewtonCooling.mo
// @ref http://book.xogeny.com/behavior/equations/physical/
model NewtonCooling "An example of Newton's law of cooling"
  parameter Real T_inf=25 "Ambient temperature";
  parameter Real T0=90 "Initial temperature";
  parameter Real h=0.7 "Convective cooling coefficient";
  parameter Real A=1.0 "Surface area";
  parameter Real m=0.1 "Mass of thermal capacitance";
  parameter Real c_p=1.2 "Specific heat";
  Real T "Temperature";
initial equation
  T = T0 "Specify initial value for T";
equation
  m*c_p*der(T) = h*A*(T_inf-T) "Newton's law of cooling";
end NewtonCooling;

Writing NewtonCooling.mo


In [5]:
# run just this file with Funz (that will check everything is ok for later parametric study)
Funz.startCalculators(1)
Funz.Run(model="Modelica",input_files="NewtonCooling.mo")

Using default input design.
Using default output expressions: ['T', 'time']
Using archive directory: /content
Over.                                                                           WARNING: An illegal reflective access operation has occurred







{'T': array([[90.        , 89.24606975, 88.50086371, 87.76422497, 87.03614242,
         86.31625075, 85.60461712, 84.90126712, 84.20620075, 83.51839021,
         82.83815209, 82.16567958, 81.50097267, 80.84403136, 80.19485566,
         79.55344557, 78.91980108, 78.29262874, 77.6720343 , 77.05872951,
         76.45271436, 75.85398886, 75.26255301, 74.6784068 , 74.10155025,
         73.53083638, 72.96541409, 72.40665508, 71.85455934, 71.30912687,
         70.77035767, 70.23825175, 69.7128091 , 69.19344532, 68.67845399,
         68.16953052, 67.66667491, 67.16988717, 66.6791673 , 66.19451528,
         65.71593113, 65.24336026, 64.77531734, 64.31279681, 63.8557396 ,
         63.40408662, 62.95777881, 62.51675707, 62.08096233, 61.65033551,
         61.22481753, 60.80434932, 60.3888718 , 59.97832588, 59.5726525 ,
         59.17179256, 58.775687  , 58.38427673, 57.99743828, 57.615127  ,
         57.23731283, 56.86393534, 56.49493407, 56.13024858, 55.76981843,
         55.41358316, 55.06148233

In [6]:
# setup main parametrized (with `$convection`) input file
%%writefile NewtonCooling.mo.par
// @ref http://book.xogeny.com/behavior/equations/physical/
model NewtonCooling "An example of Newton's law of cooling"
  parameter Real T_inf=25 "Ambient temperature";
  parameter Real T0=90 "Initial temperature";
  parameter Real h=$convection "Convective cooling coefficient";
  parameter Real A=1.0 "Surface area";
  parameter Real m=0.1 "Mass of thermal capacitance";
  parameter Real c_p=1.2 "Specific heat";
  Real T "Temperature";
initial equation
  T = T0 "Specify initial value for T";
equation
  m*c_p*der(T) = h*A*(T_inf-T) "Newton's law of cooling";
end NewtonCooling;

Writing NewtonCooling.mo.par


In [7]:
# check this input file is well read by plugin
Funz.ParseInput(model="Modelica",input_files="NewtonCooling.mo.par")

{'convection': None}

In [8]:
# launch 4 calculations for different `convection` values (0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
Funz.Run(model="Modelica",input_files="NewtonCooling.mo.par", input_variables={'convection':[0.5,0.6,0.7,0.8,0.9,1.0]}, output_expressions="min(T)")

Using model Modelica by plugin org.funz.ioplugin.BasicIOPlugin plugin from <Modelica>

  input files: /content/NewtonCooling.mo.par

  default output: T = ?

                  time = [1,2,3,4,5,6,7,8,9,10]

  syntax:  comment = *  variable = $(...)  formula  = @{...}



Using archive directory: /content
Over.                                                                           

{'calc': ['localhost:34409',
  'localhost:34409',
  'localhost:34409',
  'localhost:34409',
  'localhost:34409',
  'localhost:34409'],
 'convection': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
 'duration': [3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 'min(T)': array([25.98653154, 25.43373688, 25.19937564, 25.08828739, 25.0404579 ,
        25.01947293]),
 'state': ['done', 'done', 'done', 'done', 'done', 'done']}

In [9]:
# install Brent root finding algorithm
Funz.installDesign('Brent')

...........Funz 1.15a <build 16/02/2022 15:05>

Installed Funz design Brent


In [10]:
# find the `convection` value leading to `min(T) = 25.2` (with relative precision of 0.01 on `convection` value)
Funz.RunDesign(model="Modelica",input_files="NewtonCooling.mo.par", input_variables={'convection':"[0.5,1.0]"}, output_expressions="min(T)", 
               design="Brent", design_options={'ytarget':25.2, 'ytol':0.01})

Using archive directory: /content
Over.                                                                           

{'Root.HTML': ['in iteration number 4.<br/>the root approximation is 0.69662639165469.<br/>corresponding to the value 25.20491148180649<br/><img src="./lt4.png" width="600" height="600"/><br/>Exit due to algorithm converged<br/>'],
 'analysis': ['<HTML name="Root">in iteration number 4.<br/>the root approximation is 0.69662639165469.<br/>corresponding to the value 25.20491148180649<br/><img src="./lt4.png" width="600" height="600"/><br/>Exit due to algorithm converged<br/></HTML><root>0.69662639165469</root>'],
 'convection': array([[0.5       , 1.        , 1.        , 1.        , 0.90666178,
         0.5       , 0.90666178, 0.70333089, 0.5       , 0.70333089,
         0.69662639, 0.5       ]]),
 'min(T)': array([[25.98653154, 25.01947293, 25.01947293, 25.01947293, 25.03853105,
         25.98653154, 25.03853105, 25.19405834, 25.98653154, 25.19405834,
         25.20491148, 25.98653154]]),
 'root': ['0.69662639165469']}